In [ ]:
import pandas as pd
import os

In [ ]:
def combine_courses(folder_path):

    fall_2024_courses = {
        'Diagnostic Medicine I', 'Infectious Disease', 'Medical Sciences I',
        'Patient Assessment Clinical Reasoning I', 'Pharmacology I',
        'Principles of Medicine I'
    }
    
    all_data = []  # List to store DataFrames
    
    for filename in os.listdir(folder_path):
        # Check if the file is a CSV file
        if filename.endswith('.csv'):
            # Read the CSV file
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            
            # Extract course name by removing "CO2026_" prefix and ".csv" extension
            course_name = filename.replace('CO2026_', '').replace('.csv', '')
            course_name = course_name.replace('_', ' ')  # Replace underscores with spaces
            
            # Add a column for Course Name
            df['Course Name'] = course_name.strip()  # Remove any trailing spaces
            
            # Assign term based on course name
            term = 'Fall 2024' if course_name in fall_2024_courses else 'Spring 2025'
            
            # Add columns for Course Name and Term
            df['Term'] = term
            
            all_data.append(df)  # Append the DataFrame to the list
        
    # Combine all DataFrames into one
    combined_df = pd.concat(all_data, ignore_index=True)
    
    return combined_df

# Example usage
folder_path = 'CO2026Grades'
combined_df = combine_courses(folder_path)

# Display the resulting DataFrame
print(combined_df)


In [ ]:
combined_df

In [ ]:
combined_df['Term'].unique()

In [ ]:
combined_df['Course Name'].unique()

In [ ]:
gpa_df = pd.melt(combined_df, 
                  id_vars=['Student ID', 'Course Name'],
                  value_vars=[
                      'Overall Grade [Total Pts: up to 100 Percentage] |992319',
                      'Overall Grade [Total Pts: up to 95.25 Percentage] |989648',
                      'Overall Grade [Total Pts: up to 100 Percentage] |990539',
                      'Overall Grade [Total Pts: up to 59.04 Percentage] |976373',
                      'Overall Grade [Total Pts: up to 86.5 Percentage] |989538',
                      'Overall Grade [Total Pts: up to 94.3 Percentage] |984261',
                      'Overall Grade [Total Pts: up to 77.25 Percentage] |1044355',
                      'Overall Grade [Total Pts: up to 100 Percentage] |1048154',
                      'Overall Grade [Total Pts: up to 100 Percentage] |1048622',
                      'Overall Grade [Total Pts: up to 78.1 Percentage] |1047884',
                      'Overall Grade [Total Pts: up to 81.8625 Percentage] |1049303',
                      'Overall Grade [Total Pts: up to 72 Percentage] |1052677',
                      'Overall Grade [Total Pts: up to 100 Percentage] |1048042'
                  ],
                  var_name='variable', 
                  value_name='Overall GPA')


In [ ]:
# Rename all columns containing 'Overall Grade' to 'Overall Grade' in combined_df
combined_df.columns = [
    'Overall Grade' if 'Overall Grade' in col else col 
    for col in combined_df.columns
]

# Display updated columns
print(combined_df.columns)


In [ ]:
combined_df

In [ ]:
id_columns = ['Last Name', 'First Name', 'Username', 'Student ID', 'Last Access', 'Availability', 'Course Name', 'Term']

# Melt the DataFrame
long_df = pd.melt(combined_df, 
                  id_vars=id_columns, 
                  var_name='Exam Details', 
                  value_name='Score')



In [ ]:
long_df

In [ ]:
# Extract details from 'Exam Details' column using regex
long_df['Exam Name'] = long_df['Exam Details'].str.extract(r'^(.*?) \[', expand=True)
long_df['Total Points'] = long_df['Exam Details'].str.extract(r'Total Pts: (\d+(\.\d+)?)', expand=True)[0]
long_df['Score Type'] = long_df['Exam Details'].str.extract(r'Total Pts: .*? (\w+)', expand=True)

In [ ]:
long_df

In [ ]:
# Drop original 'Exam Details' column to tidy up
tidy_df = long_df.drop(columns=['Exam Details'])

# Optional: Fill in missing values or filter rows as needed
tidy_df = tidy_df.dropna(subset=['Exam Name', 'Score'])


In [ ]:
tidy_df['Student Name'] = tidy_df['Last Name'] + ', ' + tidy_df['First Name']
tidy_df = tidy_df.drop(columns=['Last Name', 'First Name'])

In [ ]:
tidy_df

In [ ]:
#tidy_df.to_excel('CO2026-Grades-Master.xlsx', index='False')

In [ ]:
gpa_df = gpa_df.drop(columns=['variable']).dropna()

gpa_df

In [ ]:
final_df = pd.merge(tidy_df, gpa_df, on=['Student ID', 'Course Name'])
final_df

In [ ]:
final_df.columns

In [ ]:
final_df=final_df.rename(columns={'Overall GPA':'Overall Course GPA'})

In [ ]:
prog_gpa_df = pd.read_csv('CO2026-ProgramGPAs/Student Grades.csv')
prog_gpa_df

In [ ]:
prog_gpa_df = prog_gpa_df.rename(columns={'Textbox10':'StudentID'})

In [ ]:
prog_gpa_df = prog_gpa_df[['StudentID', 'ProgramGPA']]
prog_gpa_df

In [ ]:
final_df = pd.merge(final_df, prog_gpa_df, left_on='Student ID', right_on='StudentID', how='left')

In [ ]:
final_df = final_df.drop(columns='StudentID')

In [ ]:
final_df

In [ ]:
final_df['Course Name'] = final_df['Course Name'].str.replace('CO26 ', '')

In [ ]:
def assign_credits(x):
    if x == 'Diagnostic Medicine I':
        return 2.5
    elif x == 'Infectious Disease':
        return 3
    elif x == 'Medical Sciences I':
        return 4
    elif x == 'Patient Assessment Clinical Reasoning I':
        return 2.5
    elif x == 'Pharmacology I':
        return 2
    elif x == 'Principles of Medicine I':
        return 5
    elif x == 'Behavioral Health Mental Health Medicine': 
        return 2
    elif x == 'Diagnostic Medicine II':
        return 2.5
    elif x == 'Medical Science II':
        return 4
    elif x == 'Pharmacology II':
        return 2
    elif x == 'Population Health & Wellness':
        return 2
    elif x == 'Principles of Medicine II':
        return 7
    elif x == 'Patient Assessment Clinical Reasoning II':
        return 3
    else: 
        return 0

In [ ]:
final_df['Credits'] = final_df['Course Name'].apply(assign_credits)

In [ ]:
final_df.head()

In [ ]:
final_df.to_excel('CO2026-Live-Master-31225.xlsx')